# Fine-Tuning with DistilBERT
In this notebook I will be using the pre-trained model DistilBERT to make predictions on [Natural Language Processing with Disaster Tweets](https://www.kaggle.com/competitions/nlp-getting-started/overview). Most stuff will be done by using HuggingFace libraries, and I will be fine-tuning the model by following [this HuggingFace's tutorial](https://huggingface.co/course/chapter3/1?fw=pt).

Before we begin make sure the notebook is using GPU otherwise it will take longer to train the model: [How to Enable GPU in Kaggle](https://www.kaggle.com/code/dansbecker/running-kaggle-kernels-with-a-gpu/notebook)

***

# Load the pre-trained model and its respective tokenizer 
To use different model, simple change the checkpoint to any pre-trained text classification model available in HuggingFace. It should be noted that some model can't be directly fine-tuned using transformers API. [A list of models can be found here](https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads)

In [ ]:
from transformers import  AutoModelForSequenceClassification, AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # Define which pre-trained model we will be using
classifier = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) # Get the classifier
tokenizer = AutoTokenizer.from_pretrained(checkpoint) # Get the tokenizer

***

# Load the data and preprocess it 

In [ ]:
import pandas as pd
# Load the training data
train_path = '/kaggle/input/nlp-getting-started/train.csv'
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
# Check the first few rows
df.head()

In [ ]:
# Check the data overall
df.info()

In [ ]:
# To make this simple we will drop id, keyword, location and only keep text and target
df = df.loc[:,["text", "target"]]

In [ ]:
# Split the data into train and evaluation (stratified)
from sklearn.model_selection import train_test_split
df_train, df_eval = train_test_split(df, train_size=0.8,stratify=df.target, random_state=42) # Stratified splitting 

***

# Turn pandas dataframe into dataset
We will be using Trainer API from HuggingFace for fine-tuning, and it requires data in the form of Dataset. Therefore, we will convert our Pandas DataFrame into DataSet stored in DatasetDict

In [ ]:
from datasets import Dataset, DatasetDict
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "eval": Dataset.from_pandas(df_eval)
})

In [ ]:
# Check the datasets
print("Dataset Dict:\n", raw_datasets)
print("\n\nTrain's features:\n", raw_datasets["train"].features)
print("\n\nFirst row of Train:\n", raw_datasets["train"][0])

***

# Tokenizing
Neural network require the input to be in the form of numbers for training to take place. Therefore, we will convert our text into vector of numbers (tokens) by using tokenizer.

In [ ]:
# Tokenize the text, and truncate the text if it exceed the tokenizer maximum length. Batched=True to tokenize multiple texts at the same time.
tokenized_datasets = raw_datasets.map(lambda dataset: tokenizer(dataset['text'], truncation=True), batched=True)
print(tokenized_datasets)

In [ ]:
# Check the first row
print(tokenized_datasets["train"][0])

We want to remove text and \_\_index_level_0__ as they are not needed for our model fine-tuning. Also we will rename "target" to "labels", as Trainer API require the target to be named "labels"

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("target", "labels")
print(tokenized_datasets)

***

# Training
We will be using Trainer API from HuggingFace for training. 

In [ ]:
!pip -q install evaluate

In [ ]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate

# Padding for batch of data that will be fed into model for training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training args 
training_args = TrainingArguments("test-trainer", num_train_epochs=1, evaluation_strategy="epoch", 
                                  weight_decay=5e-4, save_strategy="no", report_to="none")

# Metric for validation error
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc") # F1 and Accuracy
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define trainer
trainer = Trainer(
    classifier,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Start the fine-tuning 
trainer.train()

The trainer report our Training Loss, Validation Loss, Accuracy and also F1

***

# Quick evaluation using classification metrics
We'll be using [sklearn.metrics.classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) to do quick evaluation 

In [ ]:
from sklearn.metrics import classification_report

# Make prediction on evaluation dataset
y_pred = trainer.predict(tokenized_datasets["eval"]).predictions
y_pred = np.argmax(y_pred, axis=-1)

# Get the true labels
y_true = tokenized_datasets["eval"]["labels"]
y_true = np.array(y_true)

# Print the classification report
print(classification_report(y_true, y_pred, digits=3))

***

# Submission

In [ ]:
# Get the test data
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
ids = df_test.id # Save ids
df_test = df_test.loc[:,["text"]] # Keep only text

# Turn the DataFrame into appropriate format
test_dataset = Dataset.from_pandas(df_test)
test_dataset = test_dataset.map(lambda dataset: tokenizer(dataset['text'], truncation=True), batched=True)
test_dataset = test_dataset.remove_columns('text')

# Get the prediction
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Turn submission into DataFrame and save into CSV files
submission = pd.DataFrame({"id":ids, "target":preds})
submission.to_csv("submission.csv", index=False)